In [ ]:
# 1️⃣ Install required system packages for lap
!apt-get install -y liblapack-dev libblas-dev libopencv-dev pkg-config

# 2️⃣ Install lap from PyPI with prebuilt wheels
!pip install cython
!pip install lapx  # this is the maintained fork with wheels

# 3️⃣ Install YOLOv8
!pip install ultralytics

# 4️⃣ Install ByteTrack dependencies
!pip install onemetric
!pip install -U git+https://github.com/ifzhang/ByteTrack.git
!pip install roboflow
!pip install ultralytics
!pip install scikit-learn tqdm
!pip install -U pip
!pip install -U ultralytics
!pip install yolox loguru cython_bbox lap --quiet

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
liblapack-dev is already the newest version (3.10.0-2ubuntu1).
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
Suggested packages:
  liblapack-doc
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  libblas-dev pkg-config
0 upgraded, 2 newly installed, 2 to remove and 35 not upgraded.
Need to get 212 kB of archives.
After this operation, 1,074 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pkg-config amd64 0.29.2-1ubuntu3 [48.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libblas-dev amd64 3.10.0-2ubuntu1 [164 kB]
Fetched 212 kB in 1s (206 kB/s)
(Reading database ... 126380 files and direc

In [ ]:

from ultralytics import YOLO
from roboflow import Roboflow

import cv2
import numpy as np
from yolox.tracker.byte_tracker import BYTETracker
import numpy as np
np.float = float

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
rf = Roboflow(api_key="91TqmIgv8w7tUi53b8Kb")
project = rf.workspace("pipeline-detection").project("pipeline-detection-copy")
version = project.version(1)
dataset1 = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to pipeline-detection-copy-1 in yolov8:: 100%|██████████| 3584/3584 [00:00<00:00, 5341.24it/s]


In [ ]:
rf = Roboflow(api_key="PA3kj1H2PlxAKb2sylWl")
project = rf.workspace("mariem-vgz4r").project("pipeline-9nae1")
version = project.version(1)
dataset2 = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to pipeline-1 in yolov8:: 100%|██████████| 2478/2478 [00:00<00:00, 7280.76it/s]


In [ ]:

rf = Roboflow(api_key="KHlvL8wWXLMrMTQSGysw")
project = rf.workspace("projects-gzqta").project("pipes-aapxp")
version = project.version(2)
dataset3 = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to pipes-2 in yolov8:: 100%|██████████| 1404/1404 [00:00<00:00, 6276.86it/s]


In [ ]:
import os
import shutil
from glob import glob

# Paths to your datasets
dataset_paths = [
    dataset1.location,  # path from Roboflow download
    dataset2.location,
    dataset3.location
]

# Create merged dataset structure
merged_dataset_path = "/content/merged_dataset"
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(merged_dataset_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(merged_dataset_path, split, "labels"), exist_ok=True)

# Merge datasets
for dataset_path in dataset_paths:
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(dataset_path, split, "images")
        lbl_dir = os.path.join(dataset_path, split, "labels")

        if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
            print(f"⚠️ Missing {split} folder in {dataset_path}, skipping...")
            continue

        img_src = glob(os.path.join(img_dir, "*"))
        lbl_src = glob(os.path.join(lbl_dir, "*"))

        if not img_src:
            print(f"⚠️ No images found in {img_dir}")
        if not lbl_src:
            print(f"⚠️ No labels found in {lbl_dir}")

        for img_file in img_src:
            shutil.copy(img_file, os.path.join(merged_dataset_path, split, "images"))
        for lbl_file in lbl_src:
            shutil.copy(lbl_file, os.path.join(merged_dataset_path, split, "labels"))

        print(f"✅ {split}: copied {len(img_src)} images and {len(lbl_src)} labels from {dataset_path}")

print("🎯 Merge complete!")

# Final count check
for split in ["train", "valid", "test"]:
    img_count = len(glob(os.path.join(merged_dataset_path, split, "images", "*")))
    lbl_count = len(glob(os.path.join(merged_dataset_path, split, "labels", "*")))
    print(f"📂 {split}: {img_count} images, {lbl_count} labels")


✅ train: copied 1563 images and 1563 labels from /content/pipeline-detection-copy-1
✅ valid: copied 149 images and 149 labels from /content/pipeline-detection-copy-1
✅ test: copied 74 images and 74 labels from /content/pipeline-detection-copy-1
✅ train: copied 1236 images and 1236 labels from /content/pipeline-1
⚠️ Missing valid folder in /content/pipeline-1, skipping...
⚠️ Missing test folder in /content/pipeline-1, skipping...
✅ train: copied 699 images and 699 labels from /content/pipes-2
⚠️ Missing valid folder in /content/pipes-2, skipping...
⚠️ Missing test folder in /content/pipes-2, skipping...
🎯 Merge complete!
📂 train: 3498 images, 3498 labels
📂 valid: 149 images, 149 labels
📂 test: 74 images, 74 labels


In [ ]:
from google.colab import drive
import shutil

# 1. Mount Google Drive
#drive.mount('/content/drive')

# 2. Define paths
#src_folder = "/content/merged_dataset"
#dst_folder = "/content/drive/MyDrive/Colab Notebooks/DataSet/Pipe1"

# 3. Copy the entire folder to Google Drive
#shutil.copytree(src_folder, dst_folder)
#print("✅ Folder copied to Google Drive at:", dst_folder)


In [ ]:
!pip install pyyaml -q
import yaml

In [ ]:
# Classes list from your datasets (must be the same in all datasets)
classes = ["pipeline"]  # change if you have multiple classes
merged_dataset_path="/content/merged_dataset"
data_yaml = {
    "path": "/content/merged_dataset",
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "names": {i: name for i, name in enumerate(classes)}
}

with open(os.path.join(merged_dataset_path, "data.yaml"), "w") as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml created at", os.path.join(merged_dataset_path, "data.yaml"))


✅ data.yaml created at /content/merged_dataset/data.yaml


In [ ]:


model = YOLO("yolov8n.pt")  # use yolov8s.pt, yolov8m.pt, etc. for bigger models

model.train(
    data=os.path.join("/content/merged_dataset", "data.yaml"),
    epochs=50,
    imgsz=640,
    batch=8
)

# Save best model path
best_model_path = model.ckpt_path
print("✅ Training complete. Best model saved at:", best_model_path)


Ultralytics 8.3.181 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretr

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2365.3±801.1 MB/s, size: 99.0 KB)


train: Scanning /content/merged_dataset/train/labels... 3498 images, 144 backgrounds, 0 corrupt: 100%|██████████| 3498/3498 [00:02<00:00, 1547.38it/s]

...

       2/50      1.27G      1.477      1.846      1.569          1        640: 100%|██████████| 438/438 [00:32<00:00, 13.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 15.50it/s]

                   all        149        150      0.396       0.54      0.405      0.174



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/50      1.27G      1.444      1.733      1.533         19        640:  95%|█████████▌| 418/438 [00:30<00:01, 13.55it/s]


KeyboardInterrupt: 

In [ ]:

from google.colab import drive

drive.mount('/content/drive')
model1 = "/content/runs/detect/train2/weights/best.pt"
drive_destination = "/content/drive/MyDrive/Colab Notebooks/Models/yolov8n_final.pt"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

drive_destination = "/content/drive/MyDrive/Colab Notebooks/Models/yolov8n_final.pt"
model1 = "/content/runs/detect/train2/weights/best.pt"

# Copy best model to Drive
shutil.copy(model1, drive_destination)
print(f"📂 Model uploaded to Google Drive at: {drive_destination}")

In [ ]:

# Path to model in Google Drive
#drive_model_path = "/content/drive/MyDrive/Colab Notebooks/Models/yolov8n.pt"

# Destination path in Colab
#colab_model_path = "/content/yolov8.pt"

# Copy model from Drive to Colab
#shutil.copy(drive_model_path, colab_model_path)
#print(f"📂 Model copied to Colab at: {colab_model_path}")

In [ ]:
from google.colab import files
uploaded = files.upload()

video_path = list(uploaded.keys())[0]


## SENT


In [ ]:
import cv2
import numpy as np
np.float = np.float64  # Patch for ByteTrack

from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker
from collections import defaultdict

class Args:
    track_thresh = 0.3
    track_buffer = 50
    match_thresh = 0.7
    frame_rate = 30
    mot20 = False

args = Args()

best_model_path = model1
model = YOLO(best_model_path)

video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
args.frame_rate = fps if fps > 0 else args.frame_rate

tracker = BYTETracker(args)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("/content/tracking.mp4", fourcc, fps, (width, height))

class_colors = {
    "crack": (0, 0, 255),
    "pipeline": (255, 0, 0),
    "leak": (0, 255, 0),
    "puddle": (0, 255, 255)
}

class_names = model.names
track_trajectories = defaultdict(lambda: {'points': [], 'class_id': None})
pipeline_path_points = []

average_lines_per_frame = []
previous_x = None
previous_start_y = None
previous_end_y = None
previous_slope = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    dets = []
    class_ids = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        score = box.conf[0].item()
        cls_id = int(box.cls[0].item())
        dets.append([x1, y1, x2, y2, score])
        class_ids.append(cls_id)

    dets = np.array(dets) if len(dets) > 0 else np.empty((0, 5))
    online_targets = tracker.update(dets, (width, height), (width, height))

    frame_mid_lines = []

    for t in online_targets:
        track_id = t.track_id
        x, y, w, h = t.tlwh
        x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)

        best_iou = 0
        best_cls_id = -1
        for i, det in enumerate(dets):
            det_box = det[:4]
            track_box = [x, y, x + w, y + h]
            xi1 = max(det_box[0], track_box[0])
            yi1 = max(det_box[1], track_box[1])
            xi2 = min(det_box[2], track_box[2])
            yi2 = min(det_box[3], track_box[3])
            inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
            det_area = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
            track_area = w * h
            iou = inter_area / (det_area + track_area - inter_area)

            if iou > best_iou:
                best_iou = iou
                best_cls_id = class_ids[i]

        if best_cls_id != -1:
            class_name = class_names[best_cls_id]
            color = class_colors.get(class_name, (255, 255, 255))

            if track_trajectories[track_id]['class_id'] is None:
                track_trajectories[track_id]['class_id'] = best_cls_id

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.putText(frame, f"ID: {track_id} ({class_name})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            midline_color = (0, 255, 255)
            if (x2 - x1) > (y2 - y1):
                mid_y = (y1 + y2) // 2
                start_point = (x1, mid_y)
                end_point = (x2, mid_y)
            else:
                mid_x = (x1 + x2) // 2
                start_point = (mid_x, y1)
                end_point = (mid_x, y2)

            cv2.line(frame, start_point, end_point, midline_color, 2)
            frame_mid_lines.append((start_point, end_point))

            center = (int(x + w / 2), int(y + h / 2))
            track_trajectories[track_id]['points'].append(center)

            if class_name == "pipeline":
                pipeline_path_points.append(center)

    # Compute average middle line
    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        # Calculate slope & direction
        if previous_x is not None:
            delta_x = avg_start[0] - previous_x
            previous_slope = delta_x / (avg_end[1] - avg_start[1]) if (avg_end[1] - avg_start[1]) != 0 else 0
            if delta_x > 0:
                direction = "Right"
            elif delta_x < 0:
                direction = "Left"
            else:
                direction = "Straight"
        else:
            direction = "Unknown"
            previous_slope = 0

        previous_x = avg_start[0]
        previous_start_y = avg_start[1]
        previous_end_y = avg_end[1]

        cv2.line(frame, avg_start, avg_end, (255, 255, 255), 3)
        average_lines_per_frame.append((avg_start, avg_end, direction, previous_slope))
        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: Start {avg_start}, End {avg_end}, "
              f"Slope {previous_slope:.3f}, Direction {direction}")

    else:
        # Predict next line if YOLO fails
        if previous_x is not None:
            predicted_x = int(previous_x + previous_slope * (previous_end_y - previous_start_y))
            predicted_start = (predicted_x, previous_start_y)
            predicted_end = (predicted_x, previous_end_y)
            cv2.line(frame, predicted_start, predicted_end, (200, 200, 255), 3)  # light blue
            average_lines_per_frame.append((predicted_start, predicted_end, direction, previous_slope))
            print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: PREDICTED Start {predicted_start}, End {predicted_end}, "
                  f"Slope {previous_slope:.3f}, Direction {direction}")
            previous_x = predicted_x  # update for next prediction

    out.write(frame)

cap.release()
out.release()

print("✅ Tracking complete. Output saved at /content/tracking.mp4")

# Save average line coordinates including slope & direction
txt_path = "/content/average_pipeline_path.txt"
with open(txt_path, "w") as f:
    for idx, (start, end, direction, slope) in enumerate(average_lines_per_frame, 1):
        f.write(f"Frame {idx}: Start {start}, End {end}, Direction {direction}, Slope {slope:.3f}\n")

print(f"Average pipeline path saved to {txt_path}")


In [ ]:
#!/usr/bin/env python3
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker
from collections import defaultdict

np.float = np.float64  # Patch for ByteTrack

class Args:
    track_thresh = 0.3
    track_buffer = 50
    match_thresh = 0.7
    frame_rate = 30
    mot20 = False

args = Args()

best_model_path = "/content/runs/detect/train2/weights/best.pt"
model = YOLO(best_model_path)

video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"
output_video_path = "tracking.mp4"
output_txt_path = "average_pipeline_path1.txt"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
args.frame_rate = fps if fps > 0 else args.frame_rate

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

tracker = BYTETracker(args)

class_colors = {
    "crack": (0, 0, 255),
    "pipeline": (255, 0, 0),
    "leak": (0, 255, 0),
    "puddle": (0, 255, 255)
}

class_names = model.names
track_trajectories = defaultdict(lambda: {'points': [], 'class_id': None})
pipeline_path_points = []

average_lines_per_frame = []
previous_x = None
previous_start_y = None
previous_end_y = None
previous_slope = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    dets = []
    class_ids = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        score = box.conf[0].item()
        cls_id = int(box.cls[0].item())
        dets.append([x1, y1, x2, y2, score])
        class_ids.append(cls_id)

    if len(dets) > 0:
        dets_tensor = torch.from_numpy(np.array(dets)).float()
    else:
        dets_tensor = torch.empty((0, 5))

    online_targets = tracker.update(dets_tensor, (width, height), (width, height))
    frame_mid_lines = []

    for t in online_targets:
        track_id = t.track_id
        x, y, w, h = t.tlwh
        x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)

        best_iou = 0
        best_cls_id = -1
        for i, det in enumerate(dets):
            det_box = det[:4]
            track_box = [x, y, x + w, y + h]
            xi1 = max(det_box[0], track_box[0])
            yi1 = max(det_box[1], track_box[1])
            xi2 = min(det_box[2], track_box[2])
            yi2 = min(det_box[3], track_box[3])
            inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
            det_area = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
            track_area = w * h
            iou = inter_area / (det_area + track_area - inter_area)
            if iou > best_iou:
                best_iou = iou
                best_cls_id = class_ids[i]

        if best_cls_id != -1:
            class_name = class_names[best_cls_id]
            color = class_colors.get(class_name, (255, 255, 255))
            if track_trajectories[track_id]['class_id'] is None:
                track_trajectories[track_id]['class_id'] = best_cls_id

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.putText(frame, f"ID: {track_id} ({class_name})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            if (x2 - x1) > (y2 - y1):
                mid_y = (y1 + y2) // 2
                start_point = (x1, mid_y)
                end_point = (x2, mid_y)
            else:
                mid_x = (x1 + x2) // 2
                start_point = (mid_x, y1)
                end_point = (mid_x, y2)

            cv2.line(frame, start_point, end_point, (0, 255, 255), 2)
            frame_mid_lines.append((start_point, end_point))

            center = (int(x + w / 2), int(y + h / 2))
            track_trajectories[track_id]['points'].append(center)

            if class_name == "pipeline":
                pipeline_path_points.append(center)

    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        if previous_x is not None:
            delta_x = avg_start[0] - previous_x
            previous_slope = delta_x / (avg_end[1] - avg_start[1]) if (avg_end[1] - avg_start[1]) != 0 else 0
            if delta_x > 0:
                direction = "Right"
            elif delta_x < 0:
                direction = "Left"
            else:
                direction = "Straight"
        else:
            direction = "Unknown"
            previous_slope = 0

        previous_x = avg_start[0]
        previous_start_y = avg_start[1]
        previous_end_y = avg_end[1]

        cv2.line(frame, avg_start, avg_end, (255, 255, 255), 3)
        average_lines_per_frame.append((avg_start, avg_end, direction, previous_slope))
        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: Slope {previous_slope:.3f}, Direction {direction}")

    else:
        if previous_x is not None:
            predicted_x = int(previous_x + previous_slope * (previous_end_y - previous_start_y))
            predicted_start = (predicted_x, previous_start_y)
            predicted_end = (predicted_x, previous_end_y)
            cv2.line(frame, predicted_start, predicted_end, (200, 200, 255), 3)
            average_lines_per_frame.append((predicted_start, predicted_end, direction, previous_slope))
            print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: PREDICTED Slope {previous_slope:.3f}, Direction {direction}")
            previous_x = predicted_x

    out.write(frame)

cap.release()
out.release()

# Save average line coordinates
with open(output_txt_path, "w") as f:
    for idx, (start, end, direction, slope) in enumerate(average_lines_per_frame, 1):
        f.write(f"Frame {idx}: Start {start}, End {end}, Direction {direction}, Slope {slope:.3f}\n")

# Compute and print min/max slope
all_slopes = [slope for _, _, _, slope in average_lines_per_frame]
if all_slopes:
    print(f"Minimum slope: {min(all_slopes):.3f}")
    print(f"Maximum slope: {max(all_slopes):.3f}")

print("✅ Tracking complete. Output saved at", output_video_path)
print(f"Average pipeline path saved to {output_txt_path}")


In [ ]:
#!/usr/bin/env python3
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker
from collections import defaultdict

np.float = np.float64  # Patch for ByteTrack

class Args:
    track_thresh = 0.3
    track_buffer = 50
    match_thresh = 0.7
    frame_rate = 30
    mot20 = False

args = Args()

best_model_path = "/content/runs/detect/train2/weights/best.pt"
model = YOLO(best_model_path)

video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"
output_video_path = "tracking.mp4"
output_txt_path = "average_pipeline_path_smooth.txt"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
args.frame_rate = fps if fps > 0 else args.frame_rate

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

tracker = BYTETracker(args)

class_colors = {
    "crack": (0, 0, 255),
    "pipeline": (255, 0, 0),
    "leak": (0, 255, 0),
    "puddle": (0, 255, 255)
}

class_names = model.names
track_trajectories = defaultdict(lambda: {'points': [], 'class_id': None})
pipeline_path_points = []

average_lines_per_frame = []
previous_x = None
previous_start_y = None
previous_end_y = None
previous_slope = 0.0

# Parameters for smoothing and slope limit
alpha = 0.3              # smoothing factor (0 < alpha <= 1)
max_slope_change = 0.5    # maximum allowed slope change per frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    dets = []
    class_ids = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        score = box.conf[0].item()
        cls_id = int(box.cls[0].item())
        dets.append([x1, y1, x2, y2, score])
        class_ids.append(cls_id)

    if len(dets) > 0:
        dets_tensor = torch.from_numpy(np.array(dets)).float()
    else:
        dets_tensor = torch.empty((0, 5))

    online_targets = tracker.update(dets_tensor, (width, height), (width, height))
    frame_mid_lines = []

    for t in online_targets:
        track_id = t.track_id
        x, y, w, h = t.tlwh
        x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)

        best_iou = 0
        best_cls_id = -1
        for i, det in enumerate(dets):
            det_box = det[:4]
            track_box = [x, y, x + w, y + h]
            xi1 = max(det_box[0], track_box[0])
            yi1 = max(det_box[1], track_box[1])
            xi2 = min(det_box[2], track_box[2])
            yi2 = min(det_box[3], track_box[3])
            inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
            det_area = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
            track_area = w * h
            iou = inter_area / (det_area + track_area - inter_area)
            if iou > best_iou:
                best_iou = iou
                best_cls_id = class_ids[i]

        if best_cls_id != -1:
            class_name = class_names[best_cls_id]
            color = class_colors.get(class_name, (255, 255, 255))
            if track_trajectories[track_id]['class_id'] is None:
                track_trajectories[track_id]['class_id'] = best_cls_id

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.putText(frame, f"ID: {track_id} ({class_name})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            if (x2 - x1) > (y2 - y1):
                mid_y = (y1 + y2) // 2
                start_point = (x1, mid_y)
                end_point = (x2, mid_y)
            else:
                mid_x = (x1 + x2) // 2
                start_point = (mid_x, y1)
                end_point = (mid_x, y2)

            cv2.line(frame, start_point, end_point, (0, 255, 255), 2)
            frame_mid_lines.append((start_point, end_point))

            center = (int(x + w / 2), int(y + h / 2))
            track_trajectories[track_id]['points'].append(center)

            if class_name == "pipeline":
                pipeline_path_points.append(center)

    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        if previous_x is not None:
            delta_x = avg_start[0] - previous_x
            raw_slope = delta_x / (avg_end[1] - avg_start[1]) if (avg_end[1] - avg_start[1]) != 0 else 0

            # Limit slope change
            slope_diff = raw_slope - previous_slope
            if abs(slope_diff) > max_slope_change:
                raw_slope = previous_slope + np.sign(slope_diff) * max_slope_change

            # Smooth slope
            previous_slope = alpha * raw_slope + (1 - alpha) * previous_slope

            if delta_x > 0:
                direction = "Right"
            elif delta_x < 0:
                direction = "Left"
            else:
                direction = "Straight"
        else:
            direction = "Unknown"
            previous_slope = 0.0

        previous_x = avg_start[0]
        previous_start_y = avg_start[1]
        previous_end_y = avg_end[1]

        cv2.line(frame, avg_start, avg_end, (255, 255, 255), 3)
        average_lines_per_frame.append((avg_start, avg_end, direction, previous_slope))
        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: Slope {previous_slope:.3f}, Direction {direction}")

    else:
        if previous_x is not None:
            predicted_x = int(previous_x + previous_slope * (previous_end_y - previous_start_y))
            predicted_start = (predicted_x, previous_start_y)
            predicted_end = (predicted_x, previous_end_y)
            cv2.line(frame, predicted_start, predicted_end, (200, 200, 255), 3)
            average_lines_per_frame.append((predicted_start, predicted_end, direction, previous_slope))
            print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: PREDICTED Slope {previous_slope:.3f}, Direction {direction}")
            previous_x = predicted_x

    out.write(frame)

cap.release()
out.release()

with open(output_txt_path, "w") as f:
    for idx, (start, end, direction, slope) in enumerate(average_lines_per_frame, 1):
        f.write(f"Frame {idx}: Start {start}, End {end}, Direction {direction}, Slope {slope:.3f}\n")

# Print min/max slope
all_slopes = [slope for _, _, _, slope in average_lines_per_frame]
if all_slopes:
    print(f"Minimum slope: {min(all_slopes):.3f}")
    print(f"Maximum slope: {max(all_slopes):.3f}")

print("✅ Tracking complete. Output saved at", output_video_path)
print(f"Average pipeline path saved to {output_txt_path}")


## code + angle
The angle in this script is the angle of deviation of the pipeline relative to the horizontal axis in the image frame.

In [ ]:
#!/usr/bin/env python3
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker
from collections import defaultdict
import math

np.float = np.float64  # Patch for ByteTrack

class Args:
    track_thresh = 0.3
    track_buffer = 50
    match_thresh = 0.7
    frame_rate = 30
    mot20 = False

args = Args()

best_model_path = drive_destination
model = YOLO(best_model_path)

video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"
output_video_path = "tracking.mp4"
output_txt_path = "average_pipeline_path_angles.txt"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
args.frame_rate = fps if fps > 0 else args.frame_rate

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

tracker = BYTETracker(args)

class_colors = {
    "crack": (0, 0, 255),
    "pipeline": (255, 0, 0),
    "leak": (0, 255, 0),
    "puddle": (0, 255, 255)
}

class_names = model.names
track_trajectories = defaultdict(lambda: {'points': [], 'class_id': None})
pipeline_path_points = []

average_lines_per_frame = []
previous_x = None
previous_start_y = None
previous_end_y = None
previous_angle = 0  # Store angle instead of slope

def slope_to_angle(slope):
    """Convert slope to angle in degrees"""
    angle_rad = math.atan(slope)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    dets = []
    class_ids = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        score = box.conf[0].item()
        cls_id = int(box.cls[0].item())
        dets.append([x1, y1, x2, y2, score])
        class_ids.append(cls_id)

    if len(dets) > 0:
        dets_tensor = torch.from_numpy(np.array(dets)).float()
    else:
        dets_tensor = torch.empty((0, 5))

    online_targets = tracker.update(dets_tensor, (width, height), (width, height))
    frame_mid_lines = []

    for t in online_targets:
        track_id = t.track_id
        x, y, w, h = t.tlwh
        x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)

        best_iou = 0
        best_cls_id = -1
        for i, det in enumerate(dets):
            det_box = det[:4]
            track_box = [x, y, x + w, y + h]
            xi1 = max(det_box[0], track_box[0])
            yi1 = max(det_box[1], track_box[1])
            xi2 = min(det_box[2], track_box[2])
            yi2 = min(det_box[3], track_box[3])
            inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
            det_area = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
            track_area = w * h
            iou = inter_area / (det_area + track_area - inter_area)
            if iou > best_iou:
                best_iou = iou
                best_cls_id = class_ids[i]

        if best_cls_id != -1:
            class_name = class_names[best_cls_id]
            color = class_colors.get(class_name, (255, 255, 255))
            if track_trajectories[track_id]['class_id'] is None:
                track_trajectories[track_id]['class_id'] = best_cls_id

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.putText(frame, f"ID: {track_id} ({class_name})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            if (x2 - x1) > (y2 - y1):
                mid_y = (y1 + y2) // 2
                start_point = (x1, mid_y)
                end_point = (x2, mid_y)
            else:
                mid_x = (x1 + x2) // 2
                start_point = (mid_x, y1)
                end_point = (mid_x, y2)

            cv2.line(frame, start_point, end_point, (0, 255, 255), 2)
            frame_mid_lines.append((start_point, end_point))

            center = (int(x + w / 2), int(y + h / 2))
            track_trajectories[track_id]['points'].append(center)

            if class_name == "pipeline":
                pipeline_path_points.append(center)

    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        if previous_x is not None:
            delta_x = avg_start[0] - previous_x
            slope = delta_x / (avg_end[1] - avg_start[1]) if (avg_end[1] - avg_start[1]) != 0 else 0
            angle = slope_to_angle(slope)
            previous_angle = angle

            if delta_x > 0:
                direction = "Right"
            elif delta_x < 0:
                direction = "Left"
            else:
                direction = "Straight"
        else:
            direction = "Unknown"
            previous_angle = 0

        previous_x = avg_start[0]
        previous_start_y = avg_start[1]
        previous_end_y = avg_end[1]

        cv2.line(frame, avg_start, avg_end, (255, 255, 255), 3)
        average_lines_per_frame.append((avg_start, avg_end, direction, previous_angle))
        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: Angle {previous_angle:.2f}°, Direction {direction}")

    else:
        if previous_x is not None:
            predicted_x = int(previous_x + math.tan(math.radians(previous_angle)) * (previous_end_y - previous_start_y))
            predicted_start = (predicted_x, previous_start_y)
            predicted_end = (predicted_x, previous_end_y)
            cv2.line(frame, predicted_start, predicted_end, (200, 200, 255), 3)
            average_lines_per_frame.append((predicted_start, predicted_end, direction, previous_angle))
            print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: PREDICTED Angle {previous_angle:.2f}°, Direction {direction}")
            previous_x = predicted_x

    out.write(frame)

cap.release()
out.release()

# Save average line coordinates
with open(output_txt_path, "w") as f:
    for idx, (start, end, direction, angle) in enumerate(average_lines_per_frame, 1):
        f.write(f"Frame {idx}: Start {start}, End {end}, Direction {direction}, Angle {angle:.2f}°\n")

# Compute and print min/max angle
all_angles = [angle for _, _, _, angle in average_lines_per_frame]
if all_angles:
    print(f"Minimum angle: {min(all_angles):.2f}°")
    print(f"Maximum angle: {max(all_angles):.2f}°")

print("✅ Tracking complete. Output saved at", output_video_path)
print(f"Average pipeline path saved to {output_txt_path}")


✅ Tracking complete. Output saved at tracking.mp4
Average pipeline path saved to average_pipeline_path_angles.txt


###code angle with horizontal changes of angles

In [ ]:
#!/usr/bin/env python3
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker
from collections import defaultdict
import math

np.float = np.float64  # Patch for ByteTrack

class Args:
    track_thresh = 0.3
    track_buffer = 50
    match_thresh = 0.7
    frame_rate = 30
    mot20 = False

args = Args()


best_model_path = drive_destination
model = YOLO(best_model_path)

video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.04.22_f1bd593e.mp4"
output_video_path = "tracking_final.mp4"
output_txt_path = "pipeline_full_data.txt"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
args.frame_rate = fps if fps > 0 else args.frame_rate

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

tracker = BYTETracker(args)

class_colors = {
    "crack": (0, 0, 255),
    "pipeline": (255, 0, 0),
    "leak": (0, 255, 0),
    "puddle": (0, 255, 255)
}

class_names = model.names
track_trajectories = defaultdict(lambda: {'points': [], 'class_id': None})
pipeline_path_points = []

previous_avg_start = None
previous_avg_end = None

frame_data = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    dets = []
    class_ids = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        score = box.conf[0].item()
        cls_id = int(box.cls[0].item())
        dets.append([x1, y1, x2, y2, score])
        class_ids.append(cls_id)

    if len(dets) > 0:
        dets_tensor = torch.from_numpy(np.array(dets)).float()
    else:
        dets_tensor = torch.empty((0, 5))

    online_targets = tracker.update(dets_tensor, (width, height), (width, height))
    frame_mid_lines = []

    for t in online_targets:
        track_id = t.track_id
        x, y, w, h = t.tlwh
        x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)

        best_iou = 0
        best_cls_id = -1
        for i, det in enumerate(dets):
            det_box = det[:4]
            track_box = [x, y, x + w, y + h]
            xi1 = max(det_box[0], track_box[0])
            yi1 = max(det_box[1], track_box[1])
            xi2 = min(det_box[2], track_box[2])
            yi2 = min(det_box[3], track_box[3])
            inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
            det_area = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
            track_area = w * h
            iou = inter_area / (det_area + track_area - inter_area)
            if iou > best_iou:
                best_iou = iou
                best_cls_id = class_ids[i]

            if best_cls_id != -1:
                class_name = class_names[best_cls_id]
                color = class_colors.get(class_name, (255, 255, 255))

                # ✅ Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                cv2.putText(frame, f"ID: {track_id} ({class_name})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # ✅ Draw mid-line for the detection
            if (x2 - x1) > (y2 - y1):
                mid_y = (y1 + y2) // 2
                start_point = (x1, mid_y)
                end_point = (x2, mid_y)
            else:
                mid_x = (x1 + x2) // 2
                start_point = (mid_x, y1)
                end_point = (mid_x, y2)

            cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
            frame_mid_lines.append((start_point, end_point))


    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        if previous_avg_start is not None and previous_avg_end is not None:
            delta_x = avg_start[0] - previous_avg_start[0]
            delta_y = avg_end[1] - previous_avg_end[1]
            angle_deg = math.degrees(math.atan2(delta_y, delta_x)) if delta_x != 0 else 90.0 * (1 if delta_y >= 0 else -1)
        else:
            delta_x = 0
            delta_y = 0
            angle_deg = 0

        previous_avg_start = avg_start
        previous_avg_end = avg_end

        cv2.line(frame, avg_start, avg_end, (255, 255, 255), 3)
        frame_data.append((avg_start, avg_end, delta_x, delta_y, angle_deg))
        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: Start={avg_start}, End={avg_end}, Δx={delta_x}, Δy={delta_y}, Angle={angle_deg:.2f}°")

    out.write(frame)

cap.release()
out.release()

# Save full line coordinates + deviation + angles
with open(output_txt_path, "w") as f:
    for idx, (start, end, dx, dy, angle) in enumerate(frame_data, 1):
        f.write(f"Frame {idx}: Start={start}, End={end}, Δx={dx}, Δy={dy}, Angle={angle:.2f}°\n")

print("✅ Tracking complete. Output saved at", output_video_path)
print(f"Full pipeline line coordinates + deviation saved to {output_txt_path}")


Streaming output truncated to the last 5000 lines.
Speed: 2.0ms preprocess, 7.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
Frame 19: Start=(145, 63), End=(145, 533), Δx=-1, Δy=0, Angle=180.00°

0: 640x384 1 pipeline, 7.4ms
Speed: 2.3ms preprocess, 7.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
Frame 20: Start=(144, 64), End=(144, 530), Δx=-1, Δy=-3, Angle=-108.43°

0: 640x384 1 pipeline, 7.4ms
Speed: 2.0ms preprocess, 7.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
Frame 21: Start=(144, 68), End=(144, 534), Δx=0, Δy=4, Angle=90.00°

0: 640x384 1 pipeline, 7.4ms
Speed: 2.0ms preprocess, 7.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
Frame 22: Start=(144, 71), End=(144, 542), Δx=0, Δy=8, Angle=90.00°

0: 640x384 1 pipeline, 9.4ms
Speed: 2.1ms preprocess, 9.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
Frame 23: Start=(144, 74), End=(144, 542), Δx=0, Δy=0, Angle=90.00°

0: